In [1]:
%run base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
# forward dynamics
gravity = 9.81
x_next, rhs_xdd, states, u, dt, q_min, q_max ,arm_p, _ , _, _  = rig_dyn.forward_dynamics(gravity=gravity)

In [3]:
# forces and moments at the base
F_at_base = rig_dyn.get_base_forces(gravity=gravity)
F_at_base_eval = cs.Function(
    'F_at_base', [arm_ss.q, arm_ss.q_dot, arm_ss.q_ddot, arm_ss.sim_p, arm_ss.base_T], [F_at_base])

F_minus_xdd = F_at_base_eval(arm_ss.q, arm_ss.q_dot, rhs_xdd, arm_ss.sim_p, arm_ss.base_T)

In [4]:
F_next_ = cs.Function('Mnext', [states, u, arm_p, dt, q_min, q_max, arm_ss.base_T], [x_next, F_minus_xdd])
# F_next_.save("ARM_model.casadi")

In [5]:
q0 = [1.91318, 1.33339, 0.99245, 3.16073]
q0dot = [0.00751799, -0.0073626, 0.00281749, 0.0222385]
u0 = [0.0340454, -0.00441695, 0.0062739, -0.00505981]
state0 = cs.vertcat(q0, q0dot)

nextState , f_eval_base  = F_next_(state0, u0, alpha.sim_p, alpha.delta_t, alpha.joint_min, alpha.joint_max,alpha.base_T0)
print(nextState)

[1.91365, 1.3333, 0.99258, 3.16057, 0.0136181, -0.00252834, 0.00374711, -0.0050495]


In [6]:
f_eval_base

DM([-0.0341067, -2.18484, -0.00539287, 0.00016056, -0.016234, 10.519])

In [7]:
q0 = [1.91318, 1.33339, 0.99245, 3.16073]
q0dot = [0.0, 0.0, 0.0, 0.0]
u0 = [0.0, 0.0, 0.0, 0.0]
state0 = cs.vertcat(q0, q0dot)

nextState , f_eval_base  = F_next_(state0, u0, alpha.sim_p, alpha.delta_t, alpha.joint_min, alpha.joint_max,alpha.base_T0)
print(nextState)

[1.91318, 1.33338, 0.992452, 3.16073, -3.20547e-08, -0.00020363, 5.65792e-05, 5.15347e-05]


In [8]:
f_eval_base

DM([-0.0320159, -2.18003, 2.38073e-07, -4.5666e-05, -1.01317e-06, 10.5059])

In [6]:
# # c , cpp or matlab code generation for forward dynamics
# F_next_.generate("F_next_.c")
# os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")

0